In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
from scipy.stats import chisquare
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [61]:
dir_fig = 'fragile_x_tables/'

In [3]:
file_in = '/expanse/projects/sebat1/miladm/UCSD/LONG_READ_COHORT/METHYLATION/snoopsv_methyl/FMR1_ALL/methyl_data.tsv'
df = pd.read_table(file_in, sep='\t', header=0, keep_default_na=False)
df = df.loc[df.region_id.str.contains('_5UTR')].copy()
#df = df.reindex(index=[i for i in range(df.shape[0])])
#.reindex(labels=[])
print('df:')
display(df)

file_sam = '/expanse/projects/sebat1/miladm/UCSD/LONG_READ_COHORT/METHYLATION/snoopsv_methyl/FMR1_ALL/methyl_data_sample_order.txt'
with open(file_sam, 'r') as fh:
    sample_order = fh.readline().split(',')[:-1] # last one is an empty string
print(f'number of samples: {len(sample_order)}')
#print(f'sample order: {sample_order}')

df:


,chrom,pos,end,region_id,methyl_h1,methyl_h2,methyl_h0,bp_dev_h1,bp_dev_h2,bp_dev_h0,rn_h1,rn_h2,rn_h0
1,chrX,147911920,147912179,FMR1_5UTR,.;.;.;91|95|100|99|96|100|99|98|98|96|96|97|94...,.;.;.;64|20|7|2|0|0|0|0|0|0|2|0|1|0|0|0|0|0|3|...,.;0|0|0|0|0|0|0|1|2|32|27|13|7|5|46|27|29|15|3...,".;.;.;32;.;.;30;.;31,30;34;27,27,27,29;.;.;.;3...",".;.;.;31,43,30;.;.;30,28,28,31;.;30,30,32;34,2...",".;51,51,51,57,51,51,52;30;.;.;30;.;34,31;.;.;....",.;.;.;m54340U_200716_153006/27724288/ccs;.;.;m...,".;.;.;m54340U_200716_153006/75497573/ccs,m5434...",".;m54340U_210523_044800/52758275/ccs,m54340U_2..."
7,chrX,149505139,149505306,IDS_5UTR,.;.;.;8|2|2|1|0|0|0|0|0|1|0|7|1|0|3|2|1|4|0|0|...,.;.;.;62|51|80|91|75|95|89|98|99|98|97|99|99|9...,.;0|2|0|1|0|1|6|2|1|8|9|13|6|3|4|8|5|0|1|0|0|0...,".;.;.;1;.;.;.;.;.;2;.;.;.;.;0,0,2;.;.;-1,1,0;....",".;.;.;0;.;.;.;.;.;.;.;.;.;.;.;.;.;1,0,1;.;.;.;...",".;0,2;3;.;0,0;0;-1,1,0,1;0,-2,0;0,0,-1,0;0,0,1...",.;.;.;m54340U_200716_153006/128059025/ccs;.;.;...,.;.;.;m54340U_200716_153006/133105829/ccs;.;.;...,".;m54340U_200806_070905/88277013/ccs,m54340U_2..."
8,chrX,149631582,149631792,TMEM185A_5UTR,.;.;.;.;.;.;95|91|35|70|18|12|12|62|41|61|26|6...,.;.;.;.;73|60|29|4|2|7|3|1|1|1|1|0|0|0|8|17|11...,5|0|0|0|0|13|5|7|20|56|43|61|72|96|91|64|21|11...,".;.;.;.;.;.;-21,-21,-21;.;-21,-15;.;15,9;.;0;....",".;.;.;.;-21;.;.,9,0;.;-21,-21;.;-21;.;-11;.;.;...","-21,-19;-21,-21,-21,-21,-18,-21,-21,-25;.;-19;...",.;.;.;.;.;.;m54340U_210703_193517/110166254/cc...,.;.;.;.;m54340U_200716_153006/94110941/ccs;.;m...,"m54340U_200716_001732/174262380/ccs,m54340U_20..."
9,chrX,150363318,150363530,MAMLD1_5UTR,.;.;.;3|2|5|5|4|1|3|3|1|0|0|0|0|0|0|0|0|0|0|4|...,.;.;.;.;.;.;71|65|46|44|7|4|0|0|0|0|0|2|64|66|...,0|2|2|2|0|0|1|6|1|3|2|49|71|55|13|6|11|26|7|5|...,".;.;.;0;.;.;-1;.;0,0;32,33,35,29,29,34,34;1,-1...",".;.;.;.;.;.;1,3;.;.;1,2,2;0,1,3;.;1,1;.;1,0;.;...","-1;0,0,0;.;.;0,3;4,1;1,0,1,1;0,1,0,0;.;.;.;37,...",.;.;.;m54340U_200716_153006/71501671/ccs;.;.;m...,.;.;.;.;.;.;m54340U_200806_223416/98175724/ccs...,m54340U_200716_001732/24969595/ccs;m54340U_210...
10,chrX,150568622,150568662,MTM1_5UTR,".;.;.;.;.;.;.;.;.;3|13|4|2|0|0,98|99|98|95|97|...",".;.;.;.;.;.;.;.;.;100|100|100|100|100|100,45|5...","1|0|0|0|0|0,2|4|2|0|0|6,7|14|13|9|1|0,2|13|6|0...",".;.;.;.;.;.;.;.;.;0,0;0,0;.;0,0,0;.;.;.;.;0,0;...",".;.;.;.;.;.;.;.;.;0,2;0;.;0,-1;.;.;.;.;1;.;.;....","0,0,0,0,0;0,0,0;0;.;0,0;.;2,0,0,0;.;.,0,0,1,0,...",.;.;.;.;.;.;.;.;.;m54340U_200611_224317/158271...,.;.;.;.;.;.;.;.;.;m54340U_200611_224317/107350...,"m54340U_200716_001732/162859070/ccs,m54340U_20..."
11,chrX,150693397,150693530,MTMR1_5UTR,.;.;.;.;.;.;.;.;.;2|1|2|10|11|5|3|2|3|1|1|5|18...,.;.;.;.;.;.;.;.;.;.;.;.;2|0|0|5|2|5|12|23|34|4...,6|4|6|5|10|63|82|90|76|34|11|6|1|2|2|1|1|3|2|4...,".;.;.;.;.;.;.;.;.;1,0,1,0,2,0;1,2,2,1,1;.;.;.;...",".;.;.;.;.;.;.;.;.;.;.;.;-1,1,-1;.;0,2,3;.;.;0,...","1,0,0;0,0,0,0,2,0,1;.;0;0,0;0;0,0,0,1,1;0,-1,0...",.;.;.;.;.;.;.;.;.;m54340U_200611_224317/609751...,.;.;.;.;.;.;.;.;.;.;.;.;m54340U_200611_224317/...,"m54340U_200716_001732/53740679/ccs,m54340U_200..."
12,chrX,150898590,150898668,CD99L2_5UTR,.;.;.;8|7|9|52|43|81|48|25;.;.;88|47|83|82|98|...,".;.;.;2|0|0|1|0|0|0|0;.;.;0|0|0|0|0|0|0|13,62|...","0|0|1|0|1|1|6|4,1|1|1|4|3|16|46|69;0|0|0|0|1|1...",".;.;.;1;.;.;0,1;.;0;0,0,-1;0,0;.;0,0,1;.;.;.;....",".;.;.;1;.;.;0,0,0;.;.;0,0,1;2,1;.;1,0;.;.;.;.;...","0,0;0,0;.;1,3;2,0,1;.;.;-2,2,0;.;.;.;1,0;.;0;0...",.;.;.;m54340U_200716_153006/24904108/ccs;.;.;m...,.;.;.;m54340U_200716_153006/134351559/ccs;.;.;...,"m54340U_200716_001732/156107929/ccs,m54340U_20..."
13,chrX,150983336,150983376,HMGB3_5UTR,".;.;.;.;.;.;100|100|100|100|100,11|61|12|40|64...",.;.;.;.;.;.;29|50|50|41;.;93|98|98|85|95;70|67...,".;2|11|5|2|6,2|1|15|3|11,5|9|6|2|0;.;.;80|76|5...",".;.;.;.;.;.;0,1,0;.;.;0,0,0;0,0,0;.;0,0;.;.;.;...",".;.;.;.;.;.;.;.;0;0;0,0,0;.;2,0,0,0;.;0;0;.;-1...",".;0,0,0;.;.;0;0,0,0;.;0,0,0;0;0,0;.;0,0,0;.;0,...",.;.;.;.;.;.;m54340U_210703_193517/127796270/cc...,.;.;.;.;.;.;

number of samples: 270


In [4]:
df_exnd = df[['chrom' ,'pos', 'end', 'region_id']].copy()
cols = df.columns.tolist()[4:]
#print(cols)

for idx, sample in enumerate(sample_order):
    #if sample == 'REACH000450' or sample == 'REACH000452':
    #    continue
    for col in cols:
        df_exnd[f'{sample}_{col}'] = df[col].str.split(';').apply(lambda row: row[idx])
display(df_exnd)

,chrom,pos,end,region_id,REACH000026_methyl_h1,REACH000026_methyl_h2,REACH000026_methyl_h0,REACH000026_bp_dev_h1,REACH000026_bp_dev_h2,REACH000026_bp_dev_h0,...,REACH000684_rn_h0,REACH000685_methyl_h1,REACH000685_methyl_h2,REACH000685_methyl_h0,REACH000685_bp_dev_h1,REACH000685_bp_dev_h2,REACH000685_bp_dev_h0,REACH000685_rn_h1,REACH000685_rn_h2,REACH000685_rn_h0
1,chrX,147911920,147912179,FMR1_5UTR,.,.,.,.,.,.,...,.,.,.,2|0|15|0|2|4|35|71|4|0|1|7|0|10|0|2|0|0|0|0|58...,.,.,"13,17,33,29,16",.,.,"658567b8-0bb0-48a8-a85d-619f26f01f60,ec6386b7-..."
7,chrX,149505139,149505306,IDS_5UTR,.,.,.,.,.,.,...,"f73f95fe-76d3-41e0-8469-c8456874f172,6e115d22-...",.,.,11|0|0|0|0|15|0|0|0|4|89|91|2|0|0|0|1|9|93|89|...,.,.,"6,-14,-3,-4,-29,-24,-17,-8,-69",.,.,"dbdc51c7-868d-4c1e-aef8-54b4be7616b1,ee73bd80-..."
8,chrX,149631582,149631792,TMEM185A_5UTR,.,.,5|0|0|0|0|13|5|7|20|56|43|61|72|96|91|64|21|11...,.,.,"-21,-19",...,"3a23193f-a7dc-4723-82a1-80cdcf61dd63,87c620df-...",.,.,"100|4|30|7|40|97|0,0|0|0|0|2|0|0|1|0|0|0|0|2|2...",.,.,"-23,-13,-2,-8,-21,.",.,.,"0bdcf4ce-425a-4468-9796-785e0a2817f8,8d6dac54-..."
9,chrX,150363318,150363530,MAMLD1_5UTR,.,.,0|2|2|2|0|0|1|6|1|3|2|49|71|55|13|6|11|26|7|5|...,.,.,-1,...,"1fd78472-ace3-40c8-ac5e-a8177b6a7284,0a85a084-...",.,.,1|0|1|0|0|0|4|0|3|61|9|12|46|79|84|75|0|1|0|0|...,.,.,"-8,-3,-11,-2,-17,8,-5,-2",.,.,"b0ddb187-c71b-4f31-8e70-da57f39b8cff,9b1c13e4-..."
10,chrX,150568622,150568662,MTM1_5UTR,.,.,"1|0|0|0|0|0,2|4|2|0|0|6,7|14|13|9|1|0,2|13|6|0...",.,.,"0,0,0,0,0",...,"76f39933-8fcc-46eb-a59a-6a6cd09427dc,a36f05c0-...",.,.,"40|36|9|41|79|30|22|2|45,4|2|0|0|0|0|5,0|4|0|1...",.,.,"9,-1,-2,2,6",.,.,"7178be63-b143-43d2-b646-7f250eee984a,3152af87-..."
11,chrX,150693397,150693530,MTMR1_5UTR,.,.,6|4|6|5|10|63|82|90|76|34|11|6|1|2|2|1|1|3|2|4...,.,.,"1,0,0",...,"676248ca-67b4-40c0-9fc5-32e5b3373c0b,f81f4122-...",.,.,"0|9|0|1|0|0|0|1|13|0|16|0|0|2|0|5,55|70|9|2|1|...",.,.,"-10,-2,5,0,-3,3,5,-5,-16,-4",.,.,"33acaac3-8252-404b-b9e8-e6c5fde090e9,21350c97-..."
12,chrX,150898590,150898668,CD99L2_5UTR,.,.,"0|0|1|0|1|1|6|4,1|1|1|4|3|16|46|69",.,.,"0,0",...,"ca01fc96-4616-4d64-9ad7-2f89ec68a1e5,3484b9c0-...",33|9|91|41|0|2|1|6,"1|0|0|7|7|0|0|0|2,45|0|21|0|0|2|0|2","46|85,0|49|2|0|0|4|0|0,0|80|0|2|0|3|0|36,10|0",-3,"4,0","-6,-6,-5,.",5af74e3f-fde8-469d-9c44-8ca2cce771d2,"34a1e889-e5f6-4400-8f70-0d32c5afd4d6,7d1b38e0-...","1ae72adc-1d37-4141-bfa6-590e0ec211d1,46f0b406-..."
13,chrX,150983336,150983376,HMGB3_5UTR,.,.,.,.,.,.,...,"58341e7d-ff66-4acc-8aaa-0dbc00054c82,b0d7c6e3-...",.,.,"0|0|12|0,0|0|0|0|2",.,.,"-1,-6",.,.,"519f9439-750b-421f-9828-4705fb07f845,94933348-..."


In [5]:
file_cov = '/expanse/projects/sebat1/s3/data/sebat/long_reads_pipeline_results/coverages_mod.tsv'
#file_cov = '/tscc/projects/ps-sebat1/s3/data/sebat/long_reads_pipeline_results/coverages_mod.tsv'
df_cov = pd.read_table(file_cov, header=0, sep='\t')
multiplat_samples = ['REACH000236', 'REACH000530', 'REACH000531', 'REACH000532', 'REACH000436']
df_cov['SAMPLE'] = df_cov.apply(lambda row: row['SAMPLE']+'_'+row['COHORT'] if row['SAMPLE'] in multiplat_samples else row['SAMPLE'], axis=1)
df_cov_high = df_cov.loc[df_cov.SAMPLE.str.startswith('REACH000') & (df_cov.MEAN_COVERAGE > 9)]
display(df_cov)
high_cov_samples = df_cov_high.SAMPLE.tolist()
#print(high_cov_samples)

def get_plat(sample):
    try:
        ret = df_cov[df_cov.SAMPLE == sample]['COHORT'].values[0]
    except:
        print(f'problem with samples: {sample}')
        ret = ''
    return ret

def get_cov(sample):
    try:
        ret = df_cov[df_cov.SAMPLE == sample]['MEAN_COVERAGE'].values[0]
    except:
        print(f'problem with samples: {sample} in get_cov')
        ret = ''
    return ret

plat_dict = {sample: get_plat(sample) for sample in df_cov.SAMPLE.tolist()}
plat_dict['REACH000226_blood'] = plat_dict['REACH000226']
plat_dict['REACH000226_sperm'] = plat_dict['REACH000226']
#print('plat_dict:')
#print(plat_dict)

cov_dict = {sample: get_cov(sample) for sample in df_cov.SAMPLE.tolist()}
#print('cov_dict:')
#print(cov_dict)

,SAMPLE,COHORT,MEAN_COVERAGE
0,HG004,ONT,81.44
1,REACH000626,ONT,10.51
2,REACH000546,ONT,10.87
3,REACH000683,ONT,9.97
4,REACH000563,ONT,8.68
...,...,...,...
283,REACH000284,PB,3.43
284,REACH000672,PB,5.02
285,REACH000292,PB,9.00
286,REACH000066,PB,1.61


In [41]:
file_psam = '/expanse/projects/sebat1/miladm/UCSD/LONG_READ_COHORT/process_IL_LR/HWE/REACH_LR_platform.psam'
df_p = pd.read_table(file_psam, sep='\t', header=None, names = ['famid', 'sample_id', 'dad', 'mom', 'sex', 'phen'])
#print('df_p:')
#display(df_p)
ds_sex = df_p.set_index('sample_id').sex
#display(ds_sex)

# make mom and dad dictionary
sample_dad_dict = {}
sample_mom_dict = {}
for sample, dad, mom in zip(df_p.sample_id, df_p.dad, df_p.mom):
    #print(sample, dad, mom)
    sample_dad_dict[sample] = dad
    sample_mom_dict[sample] = mom
    
#print('complete trios:')
df_trios = df_p.loc[df_p.dad.isin(df_p.sample_id) & df_p.mom.isin(df_p.sample_id)]
print('df_trios:')
display(df_trios.shape)

### make a samples dataframe containing all samples in complete trios
df_samples = pd.DataFrame()
df_samples['sample_id'] = df_trios['sample_id']
df_samples['rel'] = 'C'

df_dads = pd.DataFrame()
df_dads['sample_id'] =  list(set(df_trios['dad'].tolist()))
df_dads['rel'] = 'D'
df_samples = pd.concat([df_samples, df_dads])

df_moms = pd.DataFrame()
df_moms['sample_id'] =  list(set(df_trios['mom'].tolist()))
df_moms['rel'] = 'M'
df_samples = pd.concat([df_samples, df_moms])

df_samples['sex'] = df_samples.sample_id.map(ds_sex)
df_samples['plat'] = df_samples.sample_id.map(plat_dict)
df_samples['coverage'] = df_samples.sample_id.map(cov_dict)
### add blood/sperm sections for REACH000226
row_add = pd.DataFrame({'sample_id':['REACH000226_blood', 'REACH000226_sperm'] ,'rel':['D', 'D'] ,'sex':[1, 1] ,'plat':['ONT', 'ONT'] ,'coverage':[1.92, 27.87]})
df_samples = pd.concat([df_samples, row_add])
pd.set_option('display.max_rows', 10)
print('df_samples:')
display(df_samples)

print(df_samples.sex.value_counts())
print(df_samples.plat.value_counts())
print(df_samples.rel.value_counts())

print('===============================')
df_samples_flt = df_samples.loc[(df_samples.coverage>=2)]
print('df_samples_flt:')
display(df_samples_flt.shape)

print('===============================')
df_samples_flt_female_kids = df_samples_flt.loc[(df_samples_flt.sex==2)&(df_samples_flt.rel=='C')]
print('df_samples_flt_female_kids:')
display(df_samples_flt_female_kids.shape)
print(df_samples_flt_female_kids.plat.value_counts())

print('===============================')
df_samples_flt_females = df_samples_flt.loc[(df_samples_flt.sex==2)]
print('df_samples_flt_females:')
display(df_samples_flt_females.shape)
print(df_samples_flt_females.plat.value_counts())
print(df_samples_flt_females.rel.value_counts())

print('===============================')
df_samples_flt_males = df_samples_flt.loc[(df_samples_flt.sex==1)]
print('df_samples_flt_males:')
display(df_samples_flt_males.shape)
print(df_samples_flt_males.plat.value_counts())
print(df_samples_flt_males.rel.value_counts())
print(df_samples_flt_males.sample_id.tolist())

df_trios:


(117, 6)

df_samples:


,sample_id,rel,sex,plat,coverage
10,REACH000026,C,1,PB,3.22
11,REACH000058,C,1,PB,6.19
12,REACH000065,C,1,PB,1.22
15,REACH000086,C,1,PB,2.70
18,REACH000089,C,2,PB,3.27
...,...,...,...,...,...
60,REACH000651,M,2,PB,5.75
61,REACH000195,M,2,PB,2.67
62,REACH000285,M,2,PB,3.00
0,REACH000226_blood,D,1,ONT,1.92


sex
1    139
2    106
Name: count, dtype: int64
plat
PB     142
ONT    103
Name: count, dtype: int64
rel
C    117
D     65
M     63
Name: count, dtype: int64
df_samples_flt:


...

plat
ONT    23
PB     16
Name: count, dtype: int64
df_samples_flt_females:


(93, 5)

plat
PB     47
ONT    46
Name: count, dtype: int64
rel
M    54
C    39
Name: count, dtype: int64
df_samples_flt_males:


(113, 5)

plat
PB     57
ONT    56
Name: count, dtype: int64
rel
C    62
D    51
Name: count, dtype: int64
['REACH000026', 'REACH000058', 'REACH000086', 'REACH000097', 'REACH000120', 'REACH000159', 'REACH000162', 'REACH000182', 'REACH000185', 'REACH000194', 'REACH000197', 'REACH000214', 'REACH000222', 'REACH000233', 'REACH000287', 'REACH000288', 'REACH000291', 'REACH000292', 'REACH000293', 'REACH000356', 'REACH000441', 'REACH000537', 'REACH000643', 'REACH000650', 'REACH000653', 'REACH000671', 'REACH000708', 'REACH000730', 'REACH000747', 'REACH000750', 'REACH000762', 'REACH000767', 'REACH000001', 'REACH000154', 'REACH000400', 'REACH000403', 'REACH000414', 'REACH000419', 'REACH000422', 'REACH000423', 'REACH000426', 'REACH000427', 'REACH000428', 'REACH000448', 'REACH000476', 'REACH000483', 'REACH000501', 'REACH000514', 'REACH000517', 'REACH000529', 'REACH000532_ONT', 'REACH000540', 'REACH000544', 'REACH000558', 'REACH000559', 'REACH000560', 'REACH000564', 'REACH000626', 'REACH000663', 'REACH000681'

In [38]:
meta_file = '/expanse/projects/sebat1/miladm/UCSD/LONG_READ_COHORT/REACH_sample_info.tsv'
df_meta = pd.read_table(meta_file, sep='\t', header=0)

rename_samples_dict = {'REACH000236': 'REACH000236_PB', 
                       'REACH000436': 'REACH000436_PB', 
                       'REACH000530': 'REACH000530_PB', 
                       'REACH000531': 'REACH000531_ONT', 
                       'REACH000532': 'REACH000532_ONT'}
df_meta['Sample_ID'] = df_meta.apply(lambda row: rename_samples_dict[row['Sample_ID']] 
                                     if row['Sample_ID'] in rename_samples_dict else row['Sample_ID'], axis=1)

aff_dict = {}
for sample, aff in zip(df_meta['Sample_ID'].tolist(), df_meta['Affected'].tolist()):
    aff_dict[sample] = aff

In [42]:
file_pca = '/expanse/projects/sebat1/j3guevar/SFARI/aspera-sf/SFARI_DS468588/data/master_phen_4.SBayesR.20210915.csv'

df_pca = pd.read_table(file_pca, sep=',', header=0)
cols = ['fid', 'iid', 'phen','sex','cohort','duo','family','is_eur','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10', 
        'PC1_PAT','PC2_PAT','PC3_PAT','PC4_PAT','PC5_PAT','PC6_PAT','PC7_PAT','PC8_PAT','PC9_PAT','PC10_PAT', 
        'PC1_MAT','PC2_MAT','PC3_MAT','PC4_MAT','PC5_MAT','PC6_MAT','PC7_MAT','PC8_MAT','PC9_MAT','PC10_MAT']
df_pca = df_pca[cols]

rename_samples_dict = {'REACH000236': 'REACH000236_PB', 
                       'REACH000436': 'REACH000436_PB', 
                       'REACH000530': 'REACH000530_PB', 
                       'REACH000531': 'REACH000531_ONT', 
                       'REACH000532': 'REACH000532_ONT'}
df_pca['iid'] = df_pca.apply(lambda row: rename_samples_dict[row['iid']] if row['iid'] in rename_samples_dict else row['iid'], axis=1)
#display(df_pca)

# first filter for the kids, then add parents to them
df_pca_flt = df_pca.loc[df_pca.iid.isin(df_trios.sample_id)].copy()
df_pca_flt['rel'] = 'C'
#print('df_pca_flt:')
#display(df_pca_flt)

df_parents = pd.DataFrame()
for index, row in df_pca_flt.iterrows():
    sample = row['iid']
    fid = row['fid']
    phen = row['phen']
    is_eur = row['is_eur']
    pc_mat = [row[f'PC{x}_MAT'] for x in range(1,11)]
    pc_pat = [row[f'PC{x}_PAT'] for x in range(1,11)]
    dad = sample_dad_dict[sample]
    mom = sample_mom_dict[sample]
    
    dict_mat = {'fid': [fid], 'iid': [mom], 'phen': ['.'], 'sex': ['Female'], 'cohort': [1], 'duo': ['.'], 'is_eur': [is_eur]}
    dict_mat.update({f'PC{x}': [pc_mat[x-1]] for x in range(1,11)})
    df_mat = pd.DataFrame(dict_mat)

    dict_pat = {'fid': [fid], 'iid': [dad], 'phen': ['.'], 'sex': ['Male'], 'cohort': [1], 'duo': ['.'], 'is_eur': [is_eur]}
    dict_pat.update({f'PC{x}': [pc_pat[x-1]] for x in range(1,11)})
    df_pat = pd.DataFrame(dict_pat)
    
    if df_parents.shape[0] == 0 or mom not in df_parents.iid.tolist():
        df_parents = pd.concat([df_parents, df_mat])
    if df_parents.shape[0] == 0 or dad not in df_parents.iid.tolist():
        df_parents = pd.concat([df_parents, df_pat])

df_parents['rel'] = 'P'
#print('df_parents:')
#display(df_parents)

df_pca_flt = pd.concat([df_pca_flt, df_parents])
print('df_pca_flt:')
display(df_pca_flt)

df_pca_flt:


,fid,iid,phen,sex,cohort,duo,family,is_eur,PC1,PC2,...,PC2_MAT,PC3_MAT,PC4_MAT,PC5_MAT,PC6_MAT,PC7_MAT,PC8_MAT,PC9_MAT,PC10_MAT,rel
136,F0001,REACH000001,ASD,Male,1,trio,1.0,1,0.008627,-0.005607,...,0.000410,0.002845,-0.019312,-0.013769,-0.014384,0.007249,0.002096,0.005399,-0.014983,C
137,F0026,REACH000026,ASD,Male,1,trio,1.0,0,0.095364,-0.045230,...,-0.047780,-0.017435,0.153686,0.005292,-0.009487,-0.020773,0.034190,0.015895,-0.011925,C
139,F0058,REACH000058,ASD,Male,1,trio,1.0,1,0.002422,-0.009216,...,-0.003289,0.003757,-0.004828,0.007650,0.005633,0.032103,0.009510,0.020007,0.016383,C
140,F0065,REACH000065,ASD,Male,1,trio,1.0,0,0.252774,0.107933,...,0.049799,0.158347,0.011411,0.004728,0.007690,-0.006605,0.008261,-0.012000,-0.009906,C
146,F0078,REACH000086,ASD,Male,1,trio,2.0,0,0.044785,0.038065,...,0.080387,0.132056,0.029583,0.020156,0.007971,-0.002383,0.014468,0.006253,-0.018303,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,F0288,REACH000756,.,Male,1,.,NaN,0,0.204041,0.127559,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P
0,F0290,REACH000763,.,Female,1,.,NaN,1,-0.030630,0.003052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P
0,F0290,REACH000764,.,Male,1,.,NaN,1,-0.027327,-0.004878,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P
0,F0291,REACH000768,.,Female,1,.,NaN,0,0.091442,-0.016917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P


In [7]:
### not needed really
file_tr_gts = 'fragile_x/FMR1_repeat_gts.tsv'
df_tr_gts = pd.read_table(file_tr_gts, header=0, sep='\t')
display(df_tr_gts)

,Sample_id,GB,Z,Qual,Fam_id,sex,rel,GB_min,GB_max,repeats_min,repeats_max,Affected,Platform
0,REACH000001,29|31,0.20|0.32,0.99,F0001,1,C,29,31,29,30,Yes,ONT
1,REACH000107,0|34,-1.64|0.51,1,F0001,2,P,0,34,20,31,Yes,ONT
2,REACH000226,0|4,-1.64|-1.38,1,F0001,1,P,0,4,20,21,Yes,ONT
3,REACH000026,.,.,.,F0026,1,C,.,.,.,.,Yes,PB
4,REACH000269,.,.,.,F0026,2,P,.,.,.,.,No,PB
...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,REACH000766,.,.,.,F0290,2,C,.,.,.,.,No,PB
261,REACH000767,30|30,0.26|0.26,0.88,F0291,1,C,30,30,30,30,Yes,PB
262,REACH000768,.,.,.,F0291,2,P,.,.,.,.,No,PB
263,REACH000769,27|27,0.07|0.07,0.48,F0291,1,P,27,27,29,29,No,PB


In [8]:
### make a dataframe for each sample to represent each bp methylation in each region
#pd.set_option('display.max_rows', 10)
def get_bp_df(df_in, sample):
    df_bps = pd.DataFrame()
    read_hps = ['h1', 'h2', 'h0']
    for idx in df_in.index:
    #for idx in range(1):
        region_id = df_in.loc[idx, 'region_id']
        for read_hp in read_hps:
            ### example: 0|0|1|0|1|1|6|4,1|1|1|4|3|16|46|69
            m_region = df_in.loc[idx, f'{sample}_methyl_{read_hp}']
            #print(f'm_region: {m_region}')

            ### example bp_dev: 0,0
            bp_dev_list = df_in.loc[idx, f'{sample}_bp_dev_{read_hp}'].split(',')
            read_name_list = df_in.loc[idx, f'{sample}_rn_{read_hp}'].split(',')
            
            if m_region == '.':
                continue
                
            m_reads = m_region.split(',')
            #print(f'm_reads: {m_reads}')
    
            m_bps = []
            read_names = []
            bp_devs = []
            hp_tags = []
            tmp = [x.split('|') for x in m_reads] ### list of lists. the inner list values are the methylation probs
            for i_x, x in enumerate(tmp):
                m_bps.extend(x)
                #read_ids.extend([f'{region_id}_{read_hp}_{i_x}' for i in range(len(x))])
                read_names.extend([read_name_list[i_x] for i in range(len(x))])
                bp_devs.extend([bp_dev_list[i_x] for i in range(len(x))])
                hp_tags.extend([read_hp for i in range(len(x))])
            #print(f'm_bps: {m_bps}')
            #print(f'read_ids: {read_ids}')
    
            df_tmp = pd.DataFrame()
            df_tmp['region_id'] = [region_id for x in m_bps]
            df_tmp['methylation'] = m_bps
            df_tmp['read_name'] = read_names
            df_tmp['bp_dev'] = bp_devs
            df_tmp['hp_tag'] = hp_tags
    
            df_bps = pd.concat([df_bps, df_tmp], ignore_index=True)

    if df_bps.shape[0]>0:
        df_bps['methylation'] = df_bps['methylation'].astype(int)
        df_bps = df_bps.loc[df_bps.bp_dev!='.']
        df_bps['bp_dev'] = df_bps['bp_dev'].astype(int)
        df_bps['subject'] = [sample for i in range(df_bps.shape[0])]
        #df_bps['haplotype'] = df_bps.bp_dev.apply(lambda x: 'PM' if x>=bp_dev_thr else 'N')
        #df_bps['haplotype'] = df_bps.read_name.apply(lambda x: read_haplotype_dict[x] if x in read_haplotype_dict else 'NA')
        #df_bps = df_bps.loc[df_bps.haplotype!='NA']
        #display(df_bps)
    return df_bps

In [ ]:
### not needed anymore
#file_fm_offsp_pb = '/expanse/projects/sebat1/miladm/UCSD/LONG_READ_COHORT/METHYLATION/PB_SAMPLES/select_samples/pb_female_kids_cov_lt_2.txt'
#file_fm_offsp_ont = '/expanse/projects/sebat1/miladm/UCSD/LONG_READ_COHORT/METHYLATION/ONT_SAMPLES/01_pod5s/prior_samples/ont_female_kids.txt'
#df_sam_pb = pd.read_table(file_fm_offsp_pb, header=None)
#print(f'number of pb samples: {df_sam_pb.shape[0]}')
#df_sam_ont = pd.read_table(file_fm_offsp_ont, header=None)
#print(f'number of pb samples: {len([x for x in df_sam_ont[0].tolist() if x not in ["REACH000592", "REACH000409"]])}')
#samples = df_sam_pb[0].tolist() + [x for x in df_sam_ont[0].tolist() if x not in ['REACH000592', 'REACH000409']]
#print(f'number of samples: {len(samples)}')
#print(samples)

In [9]:
exclude_samples = ['REACH000592', 'REACH000409', 'REACH000001', 'REACH000226']
samples = [x for x in df_samples.sample_id.tolist() if x not in exclude_samples] + ['REACH000226_blood', 'REACH000226_sperm']
print(f'number of samples: {len(samples)}')
print(samples)

df_bps = pd.DataFrame()
for sample in samples:
    df_tmp = get_bp_df(df_exnd, sample)
    df_bps = pd.concat([df_bps, df_tmp], ignore_index=True)
df_bps['rel'] = df_bps.subject.map(df_samples.set_index('sample_id').rel)
df_bps['sex'] = df_bps.subject.map(df_samples.set_index('sample_id').sex)
df_bps['coverage'] = df_bps.subject.map(df_samples.set_index('sample_id').coverage)
print('df_bps:')
display(df_bps)

number of samples: 244
['REACH000026', 'REACH000058', 'REACH000065', 'REACH000086', 'REACH000089', 'REACH000092', 'REACH000095', 'REACH000097', 'REACH000120', 'REACH000159', 'REACH000162', 'REACH000182', 'REACH000185', 'REACH000194', 'REACH000197', 'REACH000214', 'REACH000217', 'REACH000219', 'REACH000222', 'REACH000224', 'REACH000229', 'REACH000232', 'REACH000233', 'REACH000246', 'REACH000284', 'REACH000287', 'REACH000288', 'REACH000291', 'REACH000292', 'REACH000293', 'REACH000316', 'REACH000353', 'REACH000356', 'REACH000364', 'REACH000367', 'REACH000441', 'REACH000450', 'REACH000453', 'REACH000525', 'REACH000528', 'REACH000537', 'REACH000630', 'REACH000640', 'REACH000643', 'REACH000644', 'REACH000650', 'REACH000653', 'REACH000654', 'REACH000671', 'REACH000674', 'REACH000701', 'REACH000705', 'REACH000708', 'REACH000727', 'REACH000730', 'REACH000747', 'REACH000750', 'REACH000751', 'REACH000754', 'REACH000762', 'REACH000765', 'REACH000766', 'REACH000767', 'REACH000770', 'REACH000154', '

,region_id,methylation,read_name,bp_dev,hp_tag,subject,rel,sex,coverage
0,TMEM185A_5UTR,5,m54340U_200716_001732/174262380/ccs,-21,h0,REACH000026,C,1,3.22
1,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,h0,REACH000026,C,1,3.22
2,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,h0,REACH000026,C,1,3.22
3,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,h0,REACH000026,C,1,3.22
4,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,h0,REACH000026,C,1,3.22
...,...,...,...,...,...,...,...,...,...
210848,HMGB3_5UTR,12,61e50e11-513c-490f-90fb-7f2473f7576d,1,h0,REACH000226_sperm,D,1,27.87
210849,HMGB3_5UTR,0,61e50e11-513c-490f-90fb-7f2473f7576d,1,h0,REACH000226_sperm,D,1,27.87
210850,HMGB3_5UTR,2,7ce83f3e-5461-49fe-9c4a-c0304d719628,-4,h0,REACH000226_sperm,D,1,27.87
210851,HMGB3_5UTR,0,7ce83f3e-5461-49fe-9c4a-c0304d719628,-4,h0,REACH000226_sperm,D,1,27.87


In [10]:
def get_status(row, bp_dev_col, ignore_region=False):
    if row['region_id'] != 'FMR1_5UTR' and not ignore_region:
        status = 'XCI (other genes)'
    else:
        status = 'NA'
        rep = round(row[bp_dev_col] / 3) + 20
        rep_thrs = [20, 30, 40]
        if rep <= rep_thrs[0]:
            status = f'repeats<{rep_thrs[0]}'
        for idx, rep_thr in enumerate(rep_thrs[:-1]):
            if rep > rep_thr and rep <= rep_thrs[idx+1]:
                status = f'{rep_thr}<repeats<{rep_thrs[idx+1]}'
        if rep > rep_thrs[-1]:
            status = f'{rep_thrs[-1]}<repeats'
    return status

In [11]:
df_bps['bp_rep_status'] = df_bps.apply(lambda row: get_status(row, 'bp_dev'), axis=1)
print('df_bps:')
display(df_bps)

df_bps:


,region_id,methylation,read_name,bp_dev,hp_tag,subject,rel,sex,coverage,bp_rep_status
0,TMEM185A_5UTR,5,m54340U_200716_001732/174262380/ccs,-21,h0,REACH000026,C,1,3.22,XCI (other genes)
1,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,h0,REACH000026,C,1,3.22,XCI (other genes)
2,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,h0,REACH000026,C,1,3.22,XCI (other genes)
3,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,h0,REACH000026,C,1,3.22,XCI (other genes)
4,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,h0,REACH000026,C,1,3.22,XCI (other genes)
...,...,...,...,...,...,...,...,...,...,...
210848,HMGB3_5UTR,12,61e50e11-513c-490f-90fb-7f2473f7576d,1,h0,REACH000226_sperm,D,1,27.87,XCI (other genes)
210849,HMGB3_5UTR,0,61e50e11-513c-490f-90fb-7f2473f7576d,1,h0,REACH000226_sperm,D,1,27.87,XCI (other genes)
210850,HMGB3_5UTR,2,7ce83f3e-5461-49fe-9c4a-c0304d719628,-4,h0,REACH000226_sperm,D,1,27.87,XCI (other genes)
210851,HMGB3_5UTR,0,7ce83f3e-5461-49fe-9c4a-c0304d719628,-4,h0,REACH000226_sperm,D,1,27.87,XCI (other genes)


In [12]:
#### correct FMR1 repeat hp_tag for REACH000563. The hp_tag is inferred from bp_dev with a bp_dev_thr
pd.set_option('display.max_rows', 10)
bp_dev_thr = 60
#df_bps.loc[(df_bps.subject=='REACH000563')&(df_bps.region_id=='FMR1_5UTR')]['hp_tag'] = df_bps.loc[((df_bps.subject=='REACH000563')&
#                                                                                                    (df_bps.region_id=='FMR1_5UTR'))].apply(lambda row: 'h1' if row['bp_dev']>bp_dev_thr else 'h2', axis=1)
def correct_hp(row, bp_dev_thr):
    hp = row['hp_tag']
    if row['subject'] == 'REACH000563' and row['region_id'] == 'FMR1_5UTR':
        if row['bp_dev'] >= bp_dev_thr:
            hp = 'h1'
        else:
            hp = 'h2'
    return hp
df_bps['hp_tag'] = df_bps.apply(lambda row: correct_hp(row, bp_dev_thr), axis=1)
print('corrected df_bps for R563 and FMR1_5UTR:')
display(df_bps.loc[(df_bps.subject=='REACH000563')&(df_bps.region_id=='FMR1_5UTR')])
print('df_bps:')
display(df_bps)

corrected df_bps for R563 and FMR1_5UTR:


,region_id,methylation,read_name,bp_dev,hp_tag,subject,rel,sex,coverage,bp_rep_status
83641,FMR1_5UTR,0,3dc80ce9-433b-4051-9a5a-52005e0d3627,86,h1,REACH000563,C,2,8.68,40<repeats
83642,FMR1_5UTR,0,3dc80ce9-433b-4051-9a5a-52005e0d3627,86,h1,REACH000563,C,2,8.68,40<repeats
83643,FMR1_5UTR,20,3dc80ce9-433b-4051-9a5a-52005e0d3627,86,h1,REACH000563,C,2,8.68,40<repeats
83644,FMR1_5UTR,97,3dc80ce9-433b-4051-9a5a-52005e0d3627,86,h1,REACH000563,C,2,8.68,40<repeats
83645,FMR1_5UTR,80,3dc80ce9-433b-4051-9a5a-52005e0d3627,86,h1,REACH000563,C,2,8.68,40<repeats
...,...,...,...,...,...,...,...,...,...,...
84040,FMR1_5UTR,21,be004428-de04-4e9a-bb2f-3f5f0e54e1a1,53,h2,REACH000563,C,2,8.68,30<repeats<40
84041,FMR1_5UTR,88,be004428-de04-4e9a-bb2f-3f5f0e54e1a1,53,h2,REACH000563,C,2,8.68,30<repeats<40
84042,FMR1_5UTR,36,be004428-de04-4e9a-bb2f-3f5f0e54e1a1,53,h2,REACH000563,C,2,8.68,30<repeats<40
84043,FMR1_5UTR,25,be004428-de04-4e9a-bb2f-3f5f0e54e1a1,53,h2,REACH000563,C,2,8.68,30<repeats<40


df_bps:


,region_id,methylation,read_name,bp_dev,hp_tag,subject,rel,sex,coverage,bp_rep_status
0,TMEM185A_5UTR,5,m54340U_200716_001732/174262380/ccs,-21,h0,REACH000026,C,1,3.22,XCI (other genes)
1,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,h0,REACH000026,C,1,3.22,XCI (other genes)
2,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,h0,REACH000026,C,1,3.22,XCI (other genes)
3,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,h0,REACH000026,C,1,3.22,XCI (other genes)
4,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,h0,REACH000026,C,1,3.22,XCI (other genes)
...,...,...,...,...,...,...,...,...,...,...
210848,HMGB3_5UTR,12,61e50e11-513c-490f-90fb-7f2473f7576d,1,h0,REACH000226_sperm,D,1,27.87,XCI (other genes)
210849,HMGB3_5UTR,0,61e50e11-513c-490f-90fb-7f2473f7576d,1,h0,REACH000226_sperm,D,1,27.87,XCI (other genes)
210850,HMGB3_5UTR,2,7ce83f3e-5461-49fe-9c4a-c0304d719628,-4,h0,REACH000226_sperm,D,1,27.87,XCI (other genes)
210851,HMGB3_5UTR,0,7ce83f3e-5461-49fe-9c4a-c0304d719628,-4,h0,REACH000226_sperm,D,1,27.87,XCI (other genes)


In [13]:
#### correct male haplotypes to hemizygous: hm
pd.set_option('display.max_rows', 10)
def correct_hp_male(row):
    hp = row['hp_tag']
    sex = row['sex']
    if sex == 1:
        hp = 'hm'
    return hp
df_bps['hp_tag'] = df_bps.apply(lambda row: correct_hp_male(row), axis=1)
print('df_bps:')
display(df_bps)

df_bps:


,region_id,methylation,read_name,bp_dev,hp_tag,subject,rel,sex,coverage,bp_rep_status
0,TMEM185A_5UTR,5,m54340U_200716_001732/174262380/ccs,-21,hm,REACH000026,C,1,3.22,XCI (other genes)
1,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,hm,REACH000026,C,1,3.22,XCI (other genes)
2,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,hm,REACH000026,C,1,3.22,XCI (other genes)
3,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,hm,REACH000026,C,1,3.22,XCI (other genes)
4,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,hm,REACH000026,C,1,3.22,XCI (other genes)
...,...,...,...,...,...,...,...,...,...,...
210848,HMGB3_5UTR,12,61e50e11-513c-490f-90fb-7f2473f7576d,1,hm,REACH000226_sperm,D,1,27.87,XCI (other genes)
210849,HMGB3_5UTR,0,61e50e11-513c-490f-90fb-7f2473f7576d,1,hm,REACH000226_sperm,D,1,27.87,XCI (other genes)
210850,HMGB3_5UTR,2,7ce83f3e-5461-49fe-9c4a-c0304d719628,-4,hm,REACH000226_sperm,D,1,27.87,XCI (other genes)
210851,HMGB3_5UTR,0,7ce83f3e-5461-49fe-9c4a-c0304d719628,-4,hm,REACH000226_sperm,D,1,27.87,XCI (other genes)


In [14]:
this_df = df_bps.loc[df_bps.region_id=='FMR1_5UTR']
#display(this_df)
hp_info_dict = {}
#for sample in samples[:1]:
for sample in samples:
    #print(sample)
    hp_info_dict[sample] = {}
    df_tmp = this_df.loc[this_df.subject==sample]
    #print(df_tmp.shape)
        
    ### reduce redundant bps to each read/hp
    collapse_1 = df_tmp[['read_name', 'hp_tag', 'bp_dev']].groupby(by=['read_name', 'hp_tag']).median()
    #print(collapse_1)
    ### take the median of bp in each hp
    collapse_2 = collapse_1.groupby('hp_tag').median()
    #print(collapse_2)
    hps = collapse_2.index.tolist()
    for hp in hps:
        hp_info_dict[sample][hp] = round(collapse_2.loc[hp,'bp_dev'])
    #print(collapse_2.index.tolist())
    #print(df_tmp[['read_name', 'hp_tag', 'bp_dev']].groupby(by=['read_name', 'hp_tag']).mean().groupby('hp_tag').median())
    #print(df_tmp[['hp_tag', 'bp_dev']].groupby(by='hp_tag').median())
print(f'number of samples: {len(hp_info_dict)}')
print(hp_info_dict)

### thi is used for long vs short plots
hp_info_dict_comp = {}
for sample in hp_info_dict:
    if 'h1' in hp_info_dict[sample] and 'h2' in hp_info_dict[sample]:
        hp_info_dict_comp[sample] = hp_info_dict[sample]
print(f'number of samples with both h1 and h2: {len(hp_info_dict_comp)}')
print(hp_info_dict_comp)

number of samples: 242
{'REACH000026': {}, 'REACH000058': {'hm': 51}, 'REACH000065': {'hm': 30}, 'REACH000086': {'hm': 30}, 'REACH000089': {'h1': 30, 'h2': 30}, 'REACH000092': {'h1': 34, 'h2': 27}, 'REACH000095': {'h0': 4}, 'REACH000097': {'hm': 30}, 'REACH000120': {'hm': 11}, 'REACH000159': {}, 'REACH000162': {}, 'REACH000182': {'hm': 30}, 'REACH000185': {'hm': 31}, 'REACH000194': {}, 'REACH000197': {}, 'REACH000214': {'hm': 30}, 'REACH000217': {'h0': 2}, 'REACH000219': {'h0': 30}, 'REACH000222': {'hm': 27}, 'REACH000224': {}, 'REACH000229': {}, 'REACH000232': {}, 'REACH000233': {'hm': 30}, 'REACH000246': {'hm': 32}, 'REACH000284': {'h0': 16}, 'REACH000287': {'hm': 15}, 'REACH000288': {'hm': 30}, 'REACH000291': {'hm': 30}, 'REACH000292': {'hm': 30}, 'REACH000293': {'hm': 9}, 'REACH000316': {'h2': 28}, 'REACH000353': {}, 'REACH000356': {'hm': 30}, 'REACH000364': {'hm': 27}, 'REACH000367': {'h0': 27}, 'REACH000441': {'hm': 51}, 'REACH000450': {}, 'REACH000453': {'h1': 33, 'h2': 90}, 'RE

In [15]:
hp_info_dict_male = {s: hp_info_dict[s] for s in hp_info_dict if s in df_samples_flt_males.sample_id.tolist()}
print(hp_info_dict_male)

{'REACH000026': {}, 'REACH000058': {'hm': 51}, 'REACH000086': {'hm': 30}, 'REACH000097': {'hm': 30}, 'REACH000120': {'hm': 11}, 'REACH000159': {}, 'REACH000162': {}, 'REACH000182': {'hm': 30}, 'REACH000185': {'hm': 31}, 'REACH000194': {}, 'REACH000197': {}, 'REACH000214': {'hm': 30}, 'REACH000222': {'hm': 27}, 'REACH000233': {'hm': 30}, 'REACH000287': {'hm': 15}, 'REACH000288': {'hm': 30}, 'REACH000291': {'hm': 30}, 'REACH000292': {'hm': 30}, 'REACH000293': {'hm': 9}, 'REACH000356': {'hm': 30}, 'REACH000441': {'hm': 51}, 'REACH000537': {'hm': 15}, 'REACH000643': {'hm': 34}, 'REACH000650': {'hm': 28}, 'REACH000653': {'hm': 54}, 'REACH000671': {'hm': 30}, 'REACH000708': {'hm': 36}, 'REACH000730': {'hm': 36}, 'REACH000747': {'hm': 34}, 'REACH000750': {}, 'REACH000762': {'hm': 35}, 'REACH000767': {'hm': 30}, 'REACH000154': {'hm': 35}, 'REACH000400': {'hm': 8}, 'REACH000403': {'hm': -4}, 'REACH000414': {'hm': 23}, 'REACH000419': {'hm': 28}, 'REACH000422': {'hm': 22}, 'REACH000423': {'hm': 2

In [16]:
def get_hp_info(row, info_dict):
    ### we set the hp info for hp_tag or h1 and h2 only. because for h0 we don't know the haplotype and the bp_dev can be wrong by taking the median of them
    this_hp = row['hp_tag']
    if this_hp == 'h0':
        return 'NA', 'NA'
    if this_hp in info_dict[row['subject']]:
        this_hp_bp_dev = info_dict[row['subject']][this_hp]
        this_hp_rep = round(this_hp_bp_dev / 3) + 20
        return this_hp_bp_dev, this_hp_rep
    else:
        ### this could happen when you are looking at other genes with a haplotype that was not present in the FMR1 gene
        ### in these cases the information for that hp in the other genes is not known, so we leave NA
        return 'NA', 'NA'

In [17]:
df_bps[['hp_bp_dev', 'hp_rep']] = df_bps.apply(lambda row: get_hp_info(row, hp_info_dict), 
                                                                                 axis=1, result_type='expand')
df_bps['gene_tag'] = df_bps.apply(lambda row: 'FMR1 UTR' if row['region_id']=='FMR1_5UTR' else 'XCI (other genes)', axis=1)
display(df_bps)
#display(df_bps.loc[df_bps.hp_bp_dev==-10000])

,region_id,methylation,read_name,bp_dev,hp_tag,subject,rel,sex,coverage,bp_rep_status,hp_bp_dev,hp_rep,gene_tag
0,TMEM185A_5UTR,5,m54340U_200716_001732/174262380/ccs,-21,hm,REACH000026,C,1,3.22,XCI (other genes),NA,NA,XCI (other genes)
1,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,hm,REACH000026,C,1,3.22,XCI (other genes),NA,NA,XCI (other genes)
2,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,hm,REACH000026,C,1,3.22,XCI (other genes),NA,NA,XCI (other genes)
3,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,hm,REACH000026,C,1,3.22,XCI (other genes),NA,NA,XCI (other genes)
4,TMEM185A_5UTR,0,m54340U_200716_001732/174262380/ccs,-21,hm,REACH000026,C,1,3.22,XCI (other genes),NA,NA,XCI (other genes)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210848,HMGB3_5UTR,12,61e50e11-513c-490f-90fb-7f2473f7576d,1,hm,REACH000226_sperm,D,1,27.87,XCI (other genes),0,20,XCI (other genes)
210849,HMGB3_5UTR,0,61e50e11-513c-490f-90fb-7f2473f7576d,1,hm,REACH000226_sperm,D,1,27.87,XCI (other genes),0,20,XCI (other genes)
210850,HMGB3_5UTR,2,7ce83f3e-5461-49fe-9c4a-c0304d719628,-4,hm,REACH000226_sperm,D,1,27.87,XCI (other genes),0,20,XCI (other genes)
210851,HMGB3_5UTR,0,7ce83f3e-5461-49fe-9c4a-c0304d719628,-4,hm,REACH000226_sperm,D,1,27.87,XCI (other genes),0,20,XCI (other genes)


In [20]:
###
### for females we require that the haplotype is h1 or h2, and we filter out h0 becuase we don't know which haplotype it is
### and taking the median will not be accurate
df_bps_fm = df_bps.loc[(df_bps.subject.isin(df_samples_flt_females.sample_id))] # &(df_bps.hp_tag!='h0')
df_bps_fm_kid = df_bps.loc[(df_bps.subject.isin(df_samples_flt_female_kids.sample_id))]
df_bps_male = df_bps.loc[df_bps.subject.isin(df_samples_flt_males.sample_id)]
df_bps_flt = df_bps.loc[df_bps.subject.isin(df_samples_flt.sample_id)]

In [21]:
###
### make tables for all subjects with at least 2x coverage
df_flt = df_bps_flt.loc[df_bps_flt.hp_bp_dev!='NA'].copy()
df_flt['hp_rep_status'] = df_flt.apply(lambda row: get_status(row, 'hp_bp_dev', ignore_region=True), axis=1)
print('df_flt:')
display(df_flt)

df_flt:


,region_id,methylation,read_name,bp_dev,hp_tag,subject,rel,sex,coverage,bp_rep_status,hp_bp_dev,hp_rep,gene_tag,hp_rep_status
194,FMR1_5UTR,0,m54340U_210523_044800/52758275/ccs,51,hm,REACH000058,C,1,6.19,30<repeats<40,51,37,FMR1 UTR,30<repeats<40
195,FMR1_5UTR,0,m54340U_210523_044800/52758275/ccs,51,hm,REACH000058,C,1,6.19,30<repeats<40,51,37,FMR1 UTR,30<repeats<40
196,FMR1_5UTR,0,m54340U_210523_044800/52758275/ccs,51,hm,REACH000058,C,1,6.19,30<repeats<40,51,37,FMR1 UTR,30<repeats<40
197,FMR1_5UTR,0,m54340U_210523_044800/52758275/ccs,51,hm,REACH000058,C,1,6.19,30<repeats<40,51,37,FMR1 UTR,30<repeats<40
198,FMR1_5UTR,0,m54340U_210523_044800/52758275/ccs,51,hm,REACH000058,C,1,6.19,30<repeats<40,51,37,FMR1 UTR,30<repeats<40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210848,HMGB3_5UTR,12,61e50e11-513c-490f-90fb-7f2473f7576d,1,hm,REACH000226_sperm,D,1,27.87,XCI (other genes),0,20,XCI (other genes),repeats<20
210849,HMGB3_5UTR,0,61e50e11-513c-490f-90fb-7f2473f7576d,1,hm,REACH000226_sperm,D,1,27.87,XCI (other genes),0,20,XCI (other genes),repeats<20
210850,HMGB3_5UTR,2,7ce83f3e-5461-49fe-9c4a-c0304d719628,-4,hm,REACH000226_sperm,D,1,27.87,XCI (other genes),0,20,XCI (other genes),repeats<20
210851,HMGB3_5UTR,0,7ce83f3e-5461-49fe-9c4a-c0304d719628,-4,hm,REACH000226_sperm,D,1,27.87,XCI (other genes),0,20,XCI (other genes),repeats<20


In [22]:
### FMR1 haplotypes
df_flt_fmr1 = df_flt.loc[df_flt.region_id=='FMR1_5UTR'].copy()
df_flt_fmr1['bp_rep'] = (df_flt_fmr1['bp_dev'] / 3).round().astype(int) + 20
print('df_flt_fmr1:')
display(df_flt_fmr1)
df_reads_fmr1 = pd.DataFrame()
df_reads_fmr1['bp_rep'] = df_flt_fmr1.groupby(by='read_name')['bp_rep'].median().astype(int) # we first get the repeats for each read (the same for each bp)
df_reads_fmr1['methylation_mean'] = df_reads_fmr1.index.map(df_flt_fmr1.groupby(by='read_name')['methylation'].mean()) # average methylation over bps for each read
df_reads_fmr1['methylation_std'] = df_reads_fmr1.index.map(df_flt_fmr1.groupby(by='read_name')['methylation'].std())
df_reads_fmr1['methylation_mean_binary'] = df_reads_fmr1['methylation_mean'].apply(lambda x: 1 if x>=50 else 0) # decide if a read is methylated or not (make binary)
df_reads_fmr1['subject'] = df_reads_fmr1.index.map(df_flt_fmr1.groupby(by='read_name')['subject'].first())
df_reads_fmr1['hp_tag'] = df_reads_fmr1.index.map(df_flt_fmr1.groupby(by='read_name')['hp_tag'].first())
df_reads_fmr1['hp_rep'] = df_reads_fmr1.index.map(df_flt_fmr1.groupby(by='read_name')['hp_rep'].first())
print('df_reads_fmr1:')
display(df_reads_fmr1)
#pd.set_option('display.max_rows', 10)
#display(df_reads_fmr1.subject.value_counts())

df_hps_fmr1 = pd.DataFrame()
df_hps_fmr1['hp_rep'] = df_reads_fmr1.groupby(by=['subject', 'hp_tag'])['hp_rep'].first()
df_hps_fmr1['num_reads'] = df_reads_fmr1.groupby(by=['subject', 'hp_tag'])['methylation_mean'].size()
df_hps_fmr1['methylation_mean'] = df_reads_fmr1.groupby(by=['subject', 'hp_tag'])['methylation_mean'].mean()
df_hps_fmr1['methylation_std'] = df_reads_fmr1.groupby(by=['subject', 'hp_tag'])['methylation_mean'].std()
df_hps_fmr1['methylation_mean_binary'] = df_reads_fmr1.groupby(by=['subject', 'hp_tag'])['methylation_mean_binary'].mean()
df_hps_fmr1['methylation_std_binary'] = df_reads_fmr1.groupby(by=['subject', 'hp_tag'])['methylation_mean_binary'].std()
df_hps_fmr1['subject'] = df_hps_fmr1.index.map(lambda x: x[0])
df_hps_fmr1['hp_tag'] = df_hps_fmr1.index.map(lambda x: x[1])
pd.set_option('display.max_rows', 10)
print('df_hps_fmr1:')
display(df_hps_fmr1)

### XCI other genes haplotypes
df_flt_xci = df_flt.loc[df_flt.region_id!='FMR1_5UTR'].copy()
df_flt_xci['bp_rep'] = (df_flt_xci['bp_dev'] / 3).round().astype(int) + 20
print('df_flt_xci:')
display(df_flt_xci)

df_reads_xci = pd.DataFrame()
df_reads_xci['bp_rep'] = df_flt_xci.groupby(by='read_name')['bp_rep'].median().astype(int) # we first get the repeats for each read (the same for each read)
df_reads_xci['methylation_mean'] = df_reads_xci.index.map(df_flt_xci.groupby(by='read_name')['methylation'].mean()) # average methylation over bps for each read
df_reads_xci['methylation_std'] = df_reads_xci.index.map(df_flt_xci.groupby(by='read_name')['methylation'].std())
df_reads_xci['methylation_mean_binary'] = df_reads_xci['methylation_mean'].apply(lambda x: 1 if x>=50 else 0) # decide if a read is methylated or not (make binary)
df_reads_xci['subject'] = df_reads_xci.index.map(df_flt_xci.groupby(by='read_name')['subject'].first())
df_reads_xci['hp_tag'] = df_reads_xci.index.map(df_flt_xci.groupby(by='read_name')['hp_tag'].first())
df_reads_xci['hp_rep'] = df_reads_xci.index.map(df_flt_xci.groupby(by='read_name')['hp_rep'].first())
print('df_reads_xci:')
display(df_reads_xci)
#display(df_reads_xci.subject.value_counts())

df_hps_xci = pd.DataFrame()
df_hps_xci['hp_rep'] = df_reads_xci.groupby(by=['subject', 'hp_tag'])['hp_rep'].first()
df_hps_xci['num_reads'] = df_reads_xci.groupby(by=['subject', 'hp_tag'])['methylation_mean'].size()
df_hps_xci['methylation_mean'] = df_reads_xci.groupby(by=['subject', 'hp_tag'])['methylation_mean'].mean()
df_hps_xci['methylation_std'] = df_reads_xci.groupby(by=['subject', 'hp_tag'])['methylation_mean'].std()
df_hps_xci['methylation_mean_binary'] = df_reads_xci.groupby(by=['subject', 'hp_tag'])['methylation_mean_binary'].mean()
df_hps_xci['methylation_std_binary'] = df_reads_xci.groupby(by=['subject', 'hp_tag'])['methylation_mean_binary'].std()
df_hps_xci['subject'] = df_hps_xci.index.map(lambda x: x[0])
df_hps_xci['hp_tag'] = df_hps_xci.index.map(lambda x: x[1])
pd.set_option('display.max_rows', 10)
print('df_hps_xci:')
display(df_hps_xci)

df_flt_fmr1:


,region_id,methylation,read_name,bp_dev,hp_tag,subject,rel,sex,coverage,bp_rep_status,hp_bp_dev,hp_rep,gene_tag,hp_rep_status,bp_rep
194,FMR1_5UTR,0,m54340U_210523_044800/52758275/ccs,51,hm,REACH000058,C,1,6.19,30<repeats<40,51,37,FMR1 UTR,30<repeats<40,37
195,FMR1_5UTR,0,m54340U_210523_044800/52758275/ccs,51,hm,REACH000058,C,1,6.19,30<repeats<40,51,37,FMR1 UTR,30<repeats<40,37
196,FMR1_5UTR,0,m54340U_210523_044800/52758275/ccs,51,hm,REACH000058,C,1,6.19,30<repeats<40,51,37,FMR1 UTR,30<repeats<40,37
197,FMR1_5UTR,0,m54340U_210523_044800/52758275/ccs,51,hm,REACH000058,C,1,6.19,30<repeats<40,51,37,FMR1 UTR,30<repeats<40,37
198,FMR1_5UTR,0,m54340U_210523_044800/52758275/ccs,51,hm,REACH000058,C,1,6.19,30<repeats<40,51,37,FMR1 UTR,30<repeats<40,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209228,FMR1_5UTR,0,1074c822-d8c7-4eb6-9210-72ca8bfe9602,-3,hm,REACH000226_sperm,D,1,27.87,repeats<20,0,20,FMR1 UTR,repeats<20,19
209229,FMR1_5UTR,46,1074c822-d8c7-4eb6-9210-72ca8bfe9602,-3,hm,REACH000226_sperm,D,1,27.87,repeats<20,0,20,FMR1 UTR,repeats<20,19
209230,FMR1_5UTR,7,1074c822-d8c7-4eb6-9210-72ca8bfe9602,-3,hm,REACH000226_sperm,D,1,27.87,repeats<20,0,20,FMR1 UTR,repeats<20,19
209231,FMR1_5UTR,0,1074c822-d8c7-4eb6-9210-72ca8bfe9602,-3,hm,REACH000226_sperm,D,1,27.87,repeats<20,0,20,FMR1 UTR,repeats<20,19


df_reads_fmr1:


,bp_rep,methylation_mean,methylation_std,methylation_mean_binary,subject,hp_tag,hp_rep
read_name,,,,,,,
00f9b098-78b4-4f41-b956-cbde27cee4a9,35,81.431034,33.684872,1,REACH000523,h2,29
022a04bb-5a93-4714-8f50-808c07dad312,18,20.133333,31.325708,0,REACH000604,hm,18
02cf1093-a6d1-4ed9-b452-89ea2c2b661d,22,10.000000,21.982511,0,REACH000626,hm,27
03498a1d-13a3-435a-9bb4-78b3a1852470,29,4.824561,12.376710,0,REACH000503,hm,27
036caaab-f5dd-4b1b-b88c-264465c960d6,23,6.693878,15.192109,0,REACH000562,hm,25
...,...,...,...,...,...,...,...
m64278e_220618_125405/16974869/ccs,30,2.719298,5.608522,0,REACH000289,h1,30
m64278e_220618_125405/2752607/ccs,30,0.035088,0.185637,0,REACH000289,h2,30
m64278e_220618_125405/28051053/ccs,30,0.192982,0.953164,0,REACH000289,h1,30


df_hps_fmr1:


...

,region_id,methylation,read_name,bp_dev,hp_tag,subject,rel,sex,coverage,bp_rep_status,hp_bp_dev,hp_rep,gene_tag,hp_rep_status,bp_rep
637,IDS_5UTR,0,m54340U_200806_070905/88277013/ccs,0,hm,REACH000058,C,1,6.19,XCI (other genes),51,37,XCI (other genes),30<repeats<40,20
638,IDS_5UTR,2,m54340U_200806_070905/88277013/ccs,0,hm,REACH000058,C,1,6.19,XCI (other genes),51,37,XCI (other genes),30<repeats<40,20
639,IDS_5UTR,0,m54340U_200806_070905/88277013/ccs,0,hm,REACH000058,C,1,6.19,XCI (other genes),51,37,XCI (other genes),30<repeats<40,20
640,IDS_5UTR,1,m54340U_200806_070905/88277013/ccs,0,hm,REACH000058,C,1,6.19,XCI (other genes),51,37,XCI (other genes),30<repeats<40,20
641,IDS_5UTR,0,m54340U_200806_070905/88277013/ccs,0,hm,REACH000058,C,1,6.19,XCI (other genes),51,37,XCI (other genes),30<repeats<40,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210848,HMGB3_5UTR,12,61e50e11-513c-490f-90fb-7f2473f7576d,1,hm,REACH000226_sperm,D,1,27.87,XCI (other genes),0,20,XCI (other genes),repeats<20,20
210849,HMGB3_5UTR,0,61e50e11-513c-490f-90fb-7f2473f7576d,1,hm,REACH000226_sperm,D,1,27.87,XCI (other genes),0,20,XCI (other genes),repeats<20,20
210850,HMGB3_5UTR,2,7ce83f3e-5461-49fe-9c4a-c0304d719628,-4,hm,REACH000226_sperm,D,1,27.87,XCI (other genes),0,20,XCI (other genes),repeats<20,19
210851,HMGB3_5UTR,0,7ce83f3e-5461-49fe-9c4a-c0304d719628,-4,hm,REACH000226_sperm,D,1,27.87,XCI (other genes),0,20,XCI (other genes),repeats<20,19


df_reads_xci:


,bp_rep,methylation_mean,methylation_std,methylation_mean_binary,subject,hp_tag,hp_rep
read_name,,,,,,,
0027fca9-122a-43ab-b29c-4625eeafb4f3,14,84.310345,32.317734,1,REACH000482,h2,20
0034a01f-16d8-4bfd-a654-cade57eb50e6,20,2.461538,6.610481,0,REACH000427,hm,28
0040855e-b99b-4eff-9f45-852cb1b377c7,20,7.875000,15.615354,0,REACH000684,hm,28
0044cef1-af81-458e-aa05-04768f4d5da3,17,3.275862,8.526834,0,REACH000476,hm,30
008b5a8e-89f5-4ee0-984d-a8a1d2296fe7,22,4.526316,11.874881,0,REACH000226_sperm,hm,20
...,...,...,...,...,...,...,...
m64278e_220704_012231/6686007/ccs,20,4.200000,4.868265,0,REACH000290,hm,29
m64278e_220704_012231/81855701/ccs,21,1.666667,1.751190,0,REACH000290,hm,29
m64278e_220704_012231/83362719/ccs,20,0.750000,1.388730,0,REACH000290,hm,29


df_hps_xci:


hp_rep  num_reads  methylation_mean  methylation_std  \
subject     hp_tag                                                         
REACH000058 hm          37         28          7.594767         8.684887   
REACH000086 hm          30          8          6.067812         9.957010   
REACH000087 h1          30          9         35.944841        39.181304   
            h2          30          7         59.863159        34.534200   
REACH000088 hm          31         18          8.140036         8.183474   
...                    ...        ...               ...              ...   
REACH000755 h1          20         15         51.713706        40.257137   
REACH000762 hm          32         16          4.585281         6.695018   
REACH000763 h2          31         14         69.680573        37.437119   
REACH000766 h2          31         18         51.360592        40.967890   
REACH000767 hm          30         26          4.172865         6.990531   

                    methylation_mean_binary  methylation_std_binary  \
subject     hp_tag                                                    
REACH000058 hm                     0.000000                0.000000   
REACH000086 hm                     0.000000                0.000000   
REACH000087 h1                     0.333333                0.500000   
            h2                     0.714286                0.487950   
REACH000088 hm                     0.000000                0.000000   
...                                     ...                     ...   
REACH000755 h1                     0.600000                0.507093   
REACH000762 hm                     0.000000                0.000000   
REACH000763 h2                     0.785714                0.425815   
REACH000766 h2                     0.611111                0.501631   
REACH000767 hm                     0.000000                0.000000   

                        subject hp_tag  
subject     hp_tag                      
REACH000058 hm      REACH000058     hm  
REACH000086 hm      REACH000086     hm  
REACH000087 h1      REACH000087     h1  
            h2      REACH000087     h2  
REACH000088 hm      REACH000088     hm  
...                         ...    ...  
REACH000755 h1      REACH000755     h1  
REACH000762 hm      REACH000762     hm  
REACH000763 h2      REACH000763     h2  
REACH000766 h2      REACH000766     h2  
REACH000767 hm      REACH000767     hm  

[218 rows x 8 columns]

In [64]:
#pd.set_option('display.max_rows', None)
cols = ['hp_rep', 'num_reads', 'methylation_mean', 'methylation_mean_binary']
#pd.set_option('display.max_rows', None)
df_merge = pd.merge(df_hps_fmr1[cols], df_hps_xci[cols], left_index=True, right_index=True, how='inner', suffixes=('_fmr1', '_xci'))
#df_merge = df_merge.loc[(df_merge.num_reads_fmr1>2)&(df_merge.num_reads_xci>2)]
df_merge['subject'] = df_merge.index.map(lambda x: x[0])
### correct subject for REACH000226
df_merge['subject'] = df_merge.subject.apply(lambda x: 'REACH000226' if x == 'REACH000226_sperm' else x)
df_merge['hp_tag'] = df_merge.index.map(lambda x: x[1])
df_merge['fid'] = df_merge.subject.map(df_pca_flt.set_index('iid').fid)
df_merge['Affected'] = df_merge.subject.map(aff_dict)
df_merge['Phenotype'] = df_merge.Affected.map({'Yes': 1, 'No': 0})
df_merge['sex'] = df_merge.subject.map(ds_sex)
df_merge['coverage'] = df_merge.subject.map(cov_dict)
df_merge['rel'] = df_merge.subject.map(df_samples.set_index('sample_id').rel)
for i in range(1,11):
    df_merge[f'PC{i}'] = df_merge.subject.map(df_pca_flt.set_index('iid')[f'PC{i}'])
cols_old_order = df_merge.columns.tolist()
cols_new_order = cols_old_order[-10-8:-10] + cols_old_order[:-10-8] + cols_old_order[-10:]
df_merge = df_merge[cols_new_order]
print('df_merge:')
display(df_merge)
pd.set_option('display.max_rows', 10)

file_name = f'{dir_fig}/methylation_table.tsv'
print(file_name)
df_merge.to_csv(file_name, sep='\t', index=False)

df_merge:


subject hp_tag    fid Affected  Phenotype  sex  \
subject     hp_tag                                                       
REACH000058 hm      REACH000058     hm  F0058      Yes          1    1   
REACH000086 hm      REACH000086     hm  F0078      Yes          1    1   
REACH000087 h1      REACH000087     h1  F0078       No          0    2   
            h2      REACH000087     h2  F0078       No          0    2   
REACH000088 hm      REACH000088     hm  F0078       No          0    1   
...                         ...    ...    ...      ...        ...  ...   
REACH000755 h1      REACH000755     h1  F0288       No          0    2   
REACH000762 hm      REACH000762     hm  F0290      Yes          1    1   
REACH000763 h2      REACH000763     h2  F0290       No          0    2   
REACH000766 h2      REACH000766     h2  F0290       No          0    2   
REACH000767 hm      REACH000767     hm  F0291      Yes          1    1   

                    coverage rel  hp_rep_fmr1  num_reads_fmr1  ...       PC1  \
subject     hp_tag                                             ...             
REACH000058 hm          6.19   C           37               7  ...  0.002422   
REACH000086 hm          2.70   C           30               1  ...  0.044785   
REACH000087 h1          2.75   M           30               1  ...  0.129968   
            h2          2.75   M           30               4  ...  0.129968   
REACH000088 hm          3.30   D           31               2  ... -0.034480   
...                      ...  ..          ...             ...  ...       ...   
REACH000755 h1          8.76   M           20               3  ...  0.106540   
REACH000762 hm          3.79   C           32               1  ... -0.027901   
REACH000763 h2          3.36   M           31               2  ... -0.030630   
REACH000766 h2          5.83   C           31               1  ... -0.028469   
REACH000767 hm          6.31   C           30               4  ...  0.087690   

                         PC2       PC3       PC4       PC5       PC6  \
subject     hp_tag                                                     
REACH000058 hm     -0.009216 -0.007762  0.073817  0.015720  0.006303   
REACH000086 hm      0.038065  0.054606  0.014648  0.018967  0.014095   
REACH000087 h1      0.080387  0.132056  0.029583  0.020156  0.007971   
            h2      0.080387  0.132056  0.029583  0.020156  0.007971   
REACH000088 hm     -0.003222 -0.007194 -0.001006  0.007012  0.020830   
...                      ...       ...       ...       ...       ...   
REACH000755 h1      0.060475  0.110081  0.040933  0.025530  0.026022   
REACH000762 hm      0.000946 -0.004413 -0.008514 -0.003313 -0.000918   
REACH000763 h2      0.003052 -0.009075 -0.014136 -0.005499 -0.008555   
REACH000766 h2     -0.005167 -0.005808 -0.009013 -0.008074  0.000038   
REACH000767 hm     -0.012273 -0.017086  0.152272 -0.022385  0.008105   

                         PC7       PC8       PC9      PC10  
subject     hp_tag                                          
REACH000058 hm      0.055403 -0.038327 -0.011381  0.032498  
REACH000086 hm     -0.008569  0.010836  0.005924 -0.012543  
REACH000087 h1     -0.002383  0.014468  0.006253 -0.018303  
            h2     -0.002383  0.014468  0.006253 -0.018303  
REACH000088 hm     -0.009793  0.015290 -0.020905 -0.000667  
...                      ...       ...       ...       ...  
REACH000755 h1      0.015113  0.014570 -0.019842 -0.013599  
REACH000762 hm     -0.001420  0.008548  0.001257 -0.000569  
REACH000763 h2      0.015644  0.004869  0.011004 -0.019684  
REACH000766 h2      0.007456  0.005629  0.002762 -0.007944  
REACH000767 hm     -0.002083  0.017964 -0.004649 -0.006845  

[218 rows x 26 columns]

fragile_x_tables//methylation_table.tsv
